In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data45317


In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. All changes under this directory will be kept even after reset. Please clean unnecessary files in time to speed up environment loading.
!ls /home/aistudio/work

music


In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, you need to use the persistence path as the following:
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [7]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可:
# Also add the following code, so that every time the environment (kernel) starts, just run the following code:
import sys
sys.path.append('/home/aistudio/external-libraries')

In [ ]:
import sys
sys.path.append('/home/aistudio/package')

import gc
import math
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import roc_auc_score
import lightgbm as lgbm

In [ ]:
train_all=pd.read_csv('/home/aistudio/data/data45317/train_val_nn.csv')
test_all=pd.read_csv('/home/aistudio/data/data45317/validtestvalnn.csv')
members=pd.read_csv('/home/aistudio/data/data45317/members_val_nn.csv')
songs=pd.read_csv('/home/aistudio/data/data45317/songs_val_nn.csv')


In [ ]:
train_all.shape+test_all.shape+members.shape+songs.shape

(5901935, 47, 1475483, 47, 34403, 139, 419839, 100)

In [ ]:
columns=members.isnull().columns
for col in columns:
    members[col].fillna(np.nanmin(members[col]),inplace=True)

In [ ]:
members.to_csv('work/music/members_val_nn.csv',index=False)
train_all.to_csv('work/music/train_val_nn.csv',index=False)
test_all.to_csv('work/music/valid_test_val_nn.csv',index=False)
songs.to_csv('work/music/songs_val_nn.csv',index=False)

In [ ]:
train_all = pd.read_csv('work/music/train_val_nn.csv')
test_all = pd.read_csv('work/music/valid_test_val_nn.csv')
members = pd.read_csv('work/music/members_val_nn.csv')
songs = pd.read_csv('work/music/songs_val_nn.csv')

In [ ]:
train_all.shape+test_all.shape+members.shape+songs.shape

(5901935, 47, 1475483, 47, 34403, 139, 419839, 100)

 分别取50%训练集和验证集数据(各约300万数据)

In [ ]:
train=train_all[math.ceil(train_all.shape[0]*0.5):]
test=test_all[0:math.ceil(train_all.shape[0]*0.5)]
train_y=train['target']
train.drop(['target'],inplace=True,axis=1)
test_y=test['target']
test.drop(['target'],inplace=True,axis=1)



/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [ ]:
train=train.merge(members,on='msno',how='left')
test=test.merge(members,on='msno',how='left')
train=train.merge(songs,on='song_id',how='left')
test=test.merge(songs,on='song_id',how='left')


In [ ]:
train.shape+test.shape

(2950967, 283, 1475483, 283)

In [ ]:

del members
del songs
gc.collect()
train_data=lgbm.Dataset(train,label=train_y,free_raw_data=True)
test_data=lgbm.Dataset(test,label=test_y,free_raw_data=True)

In [ ]:
train.shape+test.shape

(2950967, 283, 1475483, 283)

In [ ]:
params={
    'bossting_type':'gbdt',
    'objective':'binary',
    'metric':['binary_logloss','auc'],
    'learning_rate':0.1,
    'num_leaves':180,
    'max_depth':10,
    'bagging_fraction':0.8,
    'bagging_freq': 1,
    'feature_fraction':0.8,
    'min_gain_to_split': 0,
}
evals_result={}
res=lgbm.train(params,train_data,num_boost_round=6000,valid_sets=[train_data,test_data],valid_names=['train','valid'],
evals_result=evals_result,early_stopping_rounds=1000,verbose_eval=100)
best_round=np.argmax(evals_result['valid']['auc'])
train_auc=evals_result['train']['auc'][best_round]
train_loss=evals_result['train']['binary_logloss'][best_round]
val_auc=evals_result['valid']['auc'][best_round]
val_loss=evals_result['valid']['binary_logloss'][best_round]
print('best n_estimators:',best_round)
print('Training loss: %.5f, Validation loss: %.5f'%(train_loss,val_loss))
print('Training AUC: %.5f,Validation AUC: %.5f'%(train_auc,val_auc))


Training until validation scores don't improve for 1000 rounds
[100]	train's binary_logloss: 0.522958	train's auc: 0.814631	valid's binary_logloss: 0.565703	valid's auc: 0.75402
[200]	train's binary_logloss: 0.49575	train's auc: 0.838696	valid's binary_logloss: 0.560291	valid's auc: 0.760735
[300]	train's binary_logloss: 0.478032	train's auc: 0.85321	valid's binary_logloss: 0.558618	valid's auc: 0.763405
[400]	train's binary_logloss: 0.465314	train's auc: 0.863142	valid's binary_logloss: 0.558471	valid's auc: 0.764308
[500]	train's binary_logloss: 0.454432	train's auc: 0.871394	valid's binary_logloss: 0.558733	valid's auc: 0.764722
[600]	train's binary_logloss: 0.444824	train's auc: 0.87851	valid's binary_logloss: 0.559121	valid's auc: 0.764902
[700]	train's binary_logloss: 0.436097	train's auc: 0.884797	valid's binary_logloss: 0.559713	valid's auc: 0.764882
[800]	train's binary_logloss: 0.428164	train's auc: 0.890405	valid's binary_logloss: 0.56039	valid's auc: 0.764778
[900]	train's 

In [ ]:
params={
    'bossting_type':'gbdt',
    'objective':'binary',
    'metric':['binary_logloss','auc'],
    'learning_rate':0.1,
    'num_leaves':180,
    'min_data_in_leaf':1000,
    'max_depth':10,
    'bagging_fraction':0.8,
    'bagging_freq': 1,
    'feature_fraction':0.8,
    'min_gain_to_split': 0,
}
evals_result={}
res=lgbm.train(params,train_data,num_boost_round=6000,valid_sets=[train_data,test_data],valid_names=['train','valid'],
evals_result=evals_result,early_stopping_rounds=1000,verbose_eval=100)
best_round=np.argmax(evals_result['valid']['auc'])
train_auc=evals_result['train']['auc'][best_round]
train_loss=evals_result['train']['binary_logloss'][best_round]
val_auc=evals_result['valid']['auc'][best_round]
val_loss=evals_result['valid']['binary_logloss'][best_round]
print('best n_estimators:',best_round)
print('Training loss: %.5f, Validation loss: %.5f'%(train_loss,val_loss))
print('Training AUC: %.5f,Validation AUC: %.5f'%(train_auc,val_auc))

Training until validation scores don't improve for 1000 rounds
[100]	train's binary_logloss: 0.524624	train's auc: 0.812714	valid's binary_logloss: 0.565644	valid's auc: 0.75405
[200]	train's binary_logloss: 0.499166	train's auc: 0.835504	valid's binary_logloss: 0.560243	valid's auc: 0.76071
[300]	train's binary_logloss: 0.482803	train's auc: 0.848956	valid's binary_logloss: 0.558836	valid's auc: 0.763065
[400]	train's binary_logloss: 0.470888	train's auc: 0.858342	valid's binary_logloss: 0.558393	valid's auc: 0.764393
[500]	train's binary_logloss: 0.46019	train's auc: 0.866455	valid's binary_logloss: 0.558393	valid's auc: 0.765258
[600]	train's binary_logloss: 0.451097	train's auc: 0.87318	valid's binary_logloss: 0.558675	valid's auc: 0.765687
[700]	train's binary_logloss: 0.443058	train's auc: 0.878931	valid's binary_logloss: 0.559121	valid's auc: 0.765923
[800]	train's binary_logloss: 0.43552	train's auc: 0.884267	valid's binary_logloss: 0.559564	valid's auc: 0.766097
[900]	train's 

In [ ]:
params={
    'bossting_type':'gbdt',
    'objective':'binary',
    'metric':['binary_logloss','auc'],
    'learning_rate':0.1,
    'num_leaves':180,
    'min_data_in_leaf':1000,
    'max_depth':10,
    'bagging_fraction':0.8,
    'bagging_freq': 1,
    'feature_fraction':0.8,
    'min_gain_to_split': 0,
    'lambda_l1': 6,
    'lambda_l2': 2000
}
evals_result={}
res=lgbm.train(params,train_data,num_boost_round=6000,valid_sets=[train_data,test_data],valid_names=['train','valid'],
evals_result=evals_result,early_stopping_rounds=1000,verbose_eval=100)
best_round=np.argmax(evals_result['valid']['auc'])
train_auc=evals_result['train']['auc'][best_round]
train_loss=evals_result['train']['binary_logloss'][best_round]
val_auc=evals_result['valid']['auc'][best_round]
val_loss=evals_result['valid']['binary_logloss'][best_round]
print('best n_estimators:',best_round)
print('Training loss: %.5f, Validation loss: %.5f'%(train_loss,val_loss))
print('Training AUC: %.5f,Validation AUC: %.5f'%(train_auc,val_auc))

Training until validation scores don't improve for 1000 rounds
[100]	train's binary_logloss: 0.544665	train's auc: 0.79288	valid's binary_logloss: 0.571055	valid's auc: 0.748347
[200]	train's binary_logloss: 0.524206	train's auc: 0.812636	valid's binary_logloss: 0.56367	valid's auc: 0.756581
[300]	train's binary_logloss: 0.511306	train's auc: 0.824399	valid's binary_logloss: 0.560655	valid's auc: 0.760197
[400]	train's binary_logloss: 0.501418	train's auc: 0.833025	valid's binary_logloss: 0.559078	valid's auc: 0.762335
[500]	train's binary_logloss: 0.493537	train's auc: 0.839705	valid's binary_logloss: 0.558338	valid's auc: 0.763529
[600]	train's binary_logloss: 0.486725	train's auc: 0.845296	valid's binary_logloss: 0.557855	valid's auc: 0.764476
[700]	train's binary_logloss: 0.480357	train's auc: 0.850429	valid's binary_logloss: 0.557575	valid's auc: 0.765158
[800]	train's binary_logloss: 0.474597	train's auc: 0.855026	valid's binary_logloss: 0.557462	valid's auc: 0.765659
[900]	train

In [ ]:
params={
    'bossting_type':'gbdt',
    'objective':'binary',
    'metric':['binary_logloss','auc'],
    'learning_rate':0.1,
    'num_leaves':180,
    'min_data_in_leaf':1000,
    'max_depth':10,
    'bagging_fraction':0.8,
    'bagging_freq': 1,
    'feature_fraction':0.8,
    'min_gain_to_split': 0,  
     'lambda_l1': 5,
    'lambda_l2': 2000
}
evals_result={}
res=lgbm.train(params,train_data,num_boost_round=6000,valid_sets=[train_data,test_data],valid_names=['train','valid'],
evals_result=evals_result,early_stopping_rounds=1000,verbose_eval=100)
best_round=np.argmax(evals_result['valid']['auc'])
train_auc=evals_result['train']['auc'][best_round]
train_loss=evals_result['train']['binary_logloss'][best_round]
val_auc=evals_result['valid']['auc'][best_round]
val_loss=evals_result['valid']['binary_logloss'][best_round]
print('best n_estimators:',best_round)
print('Training loss: %.5f, Validation loss: %.5f'%(train_loss,val_loss))
print('Training AUC: %.5f,Validation AUC: %.5f'%(train_auc,val_auc))

Training until validation scores don't improve for 1000 rounds
[100]	train's binary_logloss: 0.544204	train's auc: 0.793483	valid's binary_logloss: 0.570987	valid's auc: 0.74852
[200]	train's binary_logloss: 0.523954	train's auc: 0.81288	valid's binary_logloss: 0.563622	valid's auc: 0.75664
[300]	train's binary_logloss: 0.511308	train's auc: 0.824392	valid's binary_logloss: 0.560759	valid's auc: 0.760075
[400]	train's binary_logloss: 0.501482	train's auc: 0.832991	valid's binary_logloss: 0.559214	valid's auc: 0.762138
[500]	train's binary_logloss: 0.493586	train's auc: 0.839686	valid's binary_logloss: 0.558331	valid's auc: 0.763475
[600]	train's binary_logloss: 0.486611	train's auc: 0.845387	valid's binary_logloss: 0.557877	valid's auc: 0.764391
[700]	train's binary_logloss: 0.480473	train's auc: 0.850301	valid's binary_logloss: 0.55759	valid's auc: 0.765107
[800]	train's binary_logloss: 0.474996	train's auc: 0.854651	valid's binary_logloss: 0.557459	valid's auc: 0.765612
[900]	train's

从以上调参结果选择最优的一组超参数：验证集上分数是0.76719
{ 'bossting_type':'gbdt',
    'objective':'binary',
    'metric':['binary_logloss','auc'],
    'learning_rate':0.1,
    'num_leaves':180,
    'min_data_in_leaf':1000,
    'max_depth':10,
    'bagging_fraction':0.8,
    'bagging_freq': 1,
    'feature_fraction':0.8,
    'min_gain_to_split': 0,
    'lambda_l1': 6,
    'lambda_l2': 2000}

In [1]:
import sys
import gc
import math
import numpy as np
import pandas as pd
import lightgbm as lgbm
from sklearn.metrics import roc_auc_score

In [2]:
# train = pd.read_csv('work/music/train.csv')
# test = pd.read_csv('work/music/test.csv')
val_test=pd.read_csv('work/music/valid_test_val_nn.csv')
val_train=pd.read_csv('work/music/train_val_nn.csv')
members = pd.read_csv('work/music/members_val_nn.csv')
songs = pd.read_csv('work/music/songs_val_nn.csv')


In [3]:
val_train.shape+val_test.shape+members.shape+songs.shape

(5901935, 47, 1475483, 47, 34403, 139, 419839, 100)

In [5]:
train=val_train[math.ceil(val_train.shape[0]*0.7):]
test=val_test[:]
test.drop(['target'],inplace=True,axis=1)
train_y=train['target']
train.drop(['target'],inplace=True,axis=1)
train=train.merge(members,on='msno',how='left')
test=test.merge(members,on='msno',how='left')
train=train.merge(songs,on='song_id',how='left')
test=test.merge(songs,on='song_id',how='left')

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [6]:
train.shape+test.shape

(1770580, 283, 1475483, 283)

In [7]:
train_data=lgbm.Dataset(train,label=train_y,free_raw_data=True)

 取训练集中最后30%的数据，进行第一次模型训练

In [8]:
params={
    'bossting_type':'gbdt',
    'objective':'binary',
    'metric':['binary_logloss','auc'],
    'learning_rate':0.1,
    'num_leaves':180,
    'min_data_in_leaf':1000,
    'max_depth':10,
    'bagging_fraction':0.8,
    'bagging_freq': 1,
    'feature_fraction':0.8,
    'min_gain_to_split': 0,
    'lambda_l1': 6,
    'lambda_l2': 2000
}
evals_result={}
gbm=lgbm.train(params,train_data,num_boost_round=5000,valid_sets=train_data,
evals_result=evals_result,verbose_eval=100)
feature_importance=pd.DataFrame({'name':gbm.feature_name(),'importance':gbm.feature_importance()}).sort_values(by='importance',ascending=False)
feature_importance.to_csv('work/feature_importance/feat_importance_A')

[100]	training's binary_logloss: 0.544013	training's auc: 0.79119
[200]	training's binary_logloss: 0.522318	training's auc: 0.812551
[300]	training's binary_logloss: 0.507018	training's auc: 0.826782
[400]	training's binary_logloss: 0.495182	training's auc: 0.837247
[500]	training's binary_logloss: 0.485786	training's auc: 0.845168
[600]	training's binary_logloss: 0.477371	training's auc: 0.852117
[700]	training's binary_logloss: 0.469926	training's auc: 0.858061
[800]	training's binary_logloss: 0.462991	training's auc: 0.863443
[900]	training's binary_logloss: 0.456611	training's auc: 0.868302
[1000]	training's binary_logloss: 0.450433	training's auc: 0.872942
[1100]	training's binary_logloss: 0.444605	training's auc: 0.877222
[1200]	training's binary_logloss: 0.439106	training's auc: 0.881202
[1300]	training's binary_logloss: 0.43385	training's auc: 0.885001
[1400]	training's binary_logloss: 0.428903	training's auc: 0.888455
[1500]	training's binary_logloss: 0.424103	training's auc: 

In [9]:
import pickle as cPickle
cPickle.dump(gbm,open('work/music_lightgbm/submit_A.pkl','wb'))

取训练集中50%-70%位置的数据，进行第二次模型训练

In [3]:
train=val_train[math.ceil(val_train.shape[0]*0.5):math.ceil(val_train.shape[0]*0.7)]
test=val_test[:]
test.drop(['target'],inplace=True,axis=1)
train_y=train['target']
train.drop(['target'],inplace=True,axis=1)
train=train.merge(members,on='msno',how='left')
test=test.merge(members,on='msno',how='left')
train=train.merge(songs,on='song_id',how='left')
test=test.merge(songs,on='song_id',how='left')

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [4]:
train_data=lgbm.Dataset(train,label=train_y,free_raw_data=True)

In [5]:
params={
    'bossting_type':'gbdt',
    'objective':'binary',
    'metric':['binary_logloss','auc'],
    'learning_rate':0.1,
    'num_leaves':180,
    'min_data_in_leaf':1000,
    'max_depth':10,
    'bagging_fraction':0.8,
    'bagging_freq': 1,
    'feature_fraction':0.8,
    'min_gain_to_split': 0,
    'lambda_l1': 6,
    'lambda_l2': 2000
}
evals_result={}
gbm=lgbm.train(params,train_data,num_boost_round=6000,valid_sets=train_data,
evals_result=evals_result,verbose_eval=100)
feature_importance=pd.DataFrame({'name':gbm.feature_name(),'importance':gbm.feature_importance()}).sort_values(by='importance',ascending=False)
feature_importance.to_csv('work/feature_importance/feat_importance_B')

[100]	training's binary_logloss: 0.536405	training's auc: 0.805599
[200]	training's binary_logloss: 0.51137	training's auc: 0.828243
[300]	training's binary_logloss: 0.494537	training's auc: 0.842348
[400]	training's binary_logloss: 0.481468	training's auc: 0.852779
[500]	training's binary_logloss: 0.470924	training's auc: 0.860728
[600]	training's binary_logloss: 0.461322	training's auc: 0.86782
[700]	training's binary_logloss: 0.452855	training's auc: 0.873909
[800]	training's binary_logloss: 0.44527	training's auc: 0.879236
[900]	training's binary_logloss: 0.438154	training's auc: 0.884125
[1000]	training's binary_logloss: 0.431605	training's auc: 0.888598
[1100]	training's binary_logloss: 0.42517	training's auc: 0.892903
[1200]	training's binary_logloss: 0.419126	training's auc: 0.896857
[1300]	training's binary_logloss: 0.413464	training's auc: 0.900512
[1400]	training's binary_logloss: 0.408027	training's auc: 0.903944
[1500]	training's binary_logloss: 0.402827	training's auc: 0.

In [8]:
import pickle as cPickle
cPickle.dump(gbm,open('work/music_lightgbm/submit_B.pkl','wb'))

In [7]:
train=val_train[math.ceil(val_train.shape[0]*0.3):math.ceil(val_train.shape[0]*0.5)]
test=val_test[:]
test.drop(['target'],inplace=True,axis=1)
train_y=train['target']
train.drop(['target'],inplace=True,axis=1)
train=train.merge(members,on='msno',how='left')
test=test.merge(members,on='msno',how='left')
train=train.merge(songs,on='song_id',how='left')
test=test.merge(songs,on='song_id',how='left')

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [11]:
train_data=lgbm.Dataset(train,label=train_y,free_raw_data=True)

In [12]:
params={
    'bossting_type':'gbdt',
    'objective':'binary',
    'metric':['binary_logloss','auc'],
    'learning_rate':0.1,
    'num_leaves':180,
    'min_data_in_leaf':1000,
    'max_depth':10,
    'bagging_fraction':0.8,
    'bagging_freq': 1,
    'feature_fraction':0.8,
    'min_gain_to_split': 0,
    'lambda_l1': 6,
    'lambda_l2': 2000
}
evals_result={}
gbm=lgbm.train(params,train_data,num_boost_round=5000,valid_sets=train_data,
evals_result=evals_result,verbose_eval=100)
feature_importance=pd.DataFrame({'name':gbm.feature_name(),'importance':gbm.feature_importance()}).sort_values(by='importance',ascending=False)
feature_importance.to_csv('work/feature_importance/feat_importance_C')

[100]	training's binary_logloss: 0.520656	training's auc: 0.818297
[200]	training's binary_logloss: 0.495253	training's auc: 0.839756
[300]	training's binary_logloss: 0.477835	training's auc: 0.853433
[400]	training's binary_logloss: 0.464786	training's auc: 0.863057
[500]	training's binary_logloss: 0.454002	training's auc: 0.870683
[600]	training's binary_logloss: 0.44466	training's auc: 0.877088
[700]	training's binary_logloss: 0.436426	training's auc: 0.882675
[800]	training's binary_logloss: 0.428838	training's auc: 0.887687
[900]	training's binary_logloss: 0.421701	training's auc: 0.892317
[1000]	training's binary_logloss: 0.415105	training's auc: 0.896537
[1100]	training's binary_logloss: 0.409001	training's auc: 0.900332
[1200]	training's binary_logloss: 0.40315	training's auc: 0.903976
[1300]	training's binary_logloss: 0.397585	training's auc: 0.907392
[1400]	training's binary_logloss: 0.392336	training's auc: 0.910536
[1500]	training's binary_logloss: 0.387373	training's auc: 

In [13]:
import pickle as cPickle
cPickle.dump(gbm,open('work/music_lightgbm/submit_C.pkl','wb'))

In [4]:
train=val_train[math.ceil(val_train.shape[0]*0.1):math.ceil(val_train.shape[0]*0.3)]
test=val_test[:]
test.drop(['target'],inplace=True,axis=1)
train_y=train['target']
train.drop(['target'],inplace=True,axis=1)
train=train.merge(members,on='msno',how='left')
test=test.merge(members,on='msno',how='left')
train=train.merge(songs,on='song_id',how='left')
test=test.merge(songs,on='song_id',how='left')

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [5]:
train_data=lgbm.Dataset(train,label=train_y,free_raw_data=True)

In [ ]:
params={
    'bossting_type':'gbdt',
    'objective':'binary',
    'metric':['binary_logloss','auc'],
    'learning_rate':0.1,
    'num_leaves':180,
    'min_data_in_leaf':1000,
    'max_depth':10,
    'bagging_fraction':0.8,
    'bagging_freq': 1,
    'feature_fraction':0.8,
    'min_gain_to_split': 0,
    'lambda_l1': 6,
    'lambda_l2': 2000
}
evals_result={}
gbm=lgbm.train(params,train_data,num_boost_round=5000,valid_sets=train_data,
evals_result=evals_result,verbose_eval=100)
feature_importance=pd.DataFrame({'name':gbm.feature_name(),'importance':gbm.feature_importance()}).sort_values(by='importance',ascending=False)
feature_importance.to_csv('work/feature_importance/feat_importance_D')

In [ ]:
import pickle as cPickle
cPickle.dump(gbm,open('work/music_lightgbm/submit_D.pkl','wb'))

In [ ]:
import pickle as cPickle
gbm_a=cPickle.load(open('work/music_lightgbm/submit_A.pkl','rb'))

In [ ]:
size_1=math.ceil(test.shape[0]*0.33)
test_1=test[0:size_1]
test_id_1=test_id[0:size_1]
test_pred_1=gbm_a.predict(test_1)
test_sub_1=pd.DataFrame({'id':tese_id_1,'target':test_pred_1})
test_sub_1.to_csv('work/music_submit/submit_A_%.5f_01.csv'%(val_auc),index=False)

In [ ]:
del test_1
del test_id_1
del test_pred_1
gc.collect()
size_2=math.ceil(test.shape[0]*0.66)
test_2=test[size_1:size_2]
test_id_2=test_id[size_1:size_2]
test_pred_2=gbm_a.predict(test_2)
test_sub_2=pd.DataFram({'id':test_id_2,'target':test_pred_2})
test_sub_2.to_csv('work/music_submit/submit_A_%.5f_02.csv'%(val_auc),index=False)

In [ ]:
del test_2
del test_id_2
del test_pred_2
gc.collect()

test_3=test[size_2]
test_id_3=test_id[size_2:]
test_pred_3=gbm_a.predict(test_3)
test_sub_3=pd.DataFrame({'id':test_id_3,'target':test_pred_3})
test_sub_3.to_csv('work/music_submit/submit_A_%.5f_03.csv'%(val_auc),index=False)

In [ ]:
test_sub=pd.concat([test_sub_1,test_sub_2,test_sub_3],axis=0)
test_sub.to_csv('work/music_submit/submit_A_%.5f_123.csv'%(val_auc),index=False)

In [ ]:
import pickle as cPickle
gbm_b=cPickle.load(open('work/music_lightgbm/submit_B.pkl','rb'))

In [ ]:
size_1=math.ceil(test.shape[0]*0.33)
test_1=test[0:size_1]
test_id_1=test_id[0:size_1]
test_pred_1=gbm_b.predict(test_1)
test_sub_1=pd.DataFrame({'id':tese_id_1,'target':test_pred_1})
test_sub_1.to_csv('work/music_submit/submit_B_%.5f_01.csv'%(val_auc),index=False)

In [ ]:
del test_1
del test_id_1
del test_pred_1
gc.collect()
size_2=math.ceil(test.shape[0]*0.66)
test_2=test[size_1:size_2]
test_id_2=test_id[size_1:size_2]
test_pred_2=gbm_b.predict(test_2)
test_sub_2=pd.DataFram({'id':test_id_2,'target':test_pred_2})
test_sub_2.to_csv('work/music_submit/submit_B_%.5f_02.csv'%(val_auc),index=False)

In [ ]:
del test_2
del test_id_2
del test_pred_2
gc.collect()

test_3=test[size_2]
test_id_3=test_id[size_2:]
test_pred_3=gbm_b.predict(test_3)
test_sub_3=pd.DataFrame({'id':test_id_3,'target':test_pred_3})
test_sub_3.to_csv('work/music_submit/submit_B_%.5f_03.csv'%(val_auc),index=False)

In [ ]:
test_sub=pd.concat([test_sub_1,test_sub_2,test_sub_3],axis=0)
test_sub.to_csv('work/music_submit/submit_B_%.5f_123.csv'%(val_auc),index=False)

In [ ]:
import pickle as cPickle
gbm_c=cPickle.load(open('work/music_lightgbm/submit_C.pkl','rb'))

In [ ]:
size_1=math.ceil(test.shape[0]*0.33)
test_1=test[0:size_1]
test_id_1=test_id[0:size_1]
test_pred_1=gbm_c.predict(test_1)
test_sub_1=pd.DataFrame({'id':tese_id_1,'target':test_pred_1})
test_sub_1.to_csv('work/music_submit/submit_C_%.5f_01.csv'%(val_auc),index=False)

In [ ]:
del test_1
del test_id_1
del test_pred_1
gc.collect()
size_2=math.ceil(test.shape[0]*0.66)
test_2=test[size_1:size_2]
test_id_2=test_id[size_1:size_2]
test_pred_2=gbm_c.predict(test_2)
test_sub_2=pd.DataFram({'id':test_id_2,'target':test_pred_2})
test_sub_2.to_csv('work/music_submit/submit_C_%.5f_02.csv'%(val_auc),index=False)

In [ ]:
del test_2
del test_id_2
del test_pred_2
gc.collect()

test_3=test[size_2]
test_id_3=test_id[size_2:]
test_pred_3=gbm_c.predict(test_3)
test_sub_3=pd.DataFrame({'id':test_id_3,'target':test_pred_3})
test_sub_3.to_csv('work/music_submit/submit_C_%.5f_03.csv'%(val_auc),index=False)

In [ ]:
test_sub=pd.concat([test_sub_1,test_sub_2,test_sub_3],axis=0)
test_sub.to_csv('work/music_submit/submit_C_%.5f_123.csv'%(val_auc),index=False)

In [ ]:
size_1=math.ceil(test.shape[0]*0.33)
test_1=test[0:size_1]
test_id_1=test_id[0:size_1]
test_pred_1=gbm_d.predict(test_1)
test_sub_1=pd.DataFrame({'id':tese_id_1,'target':test_pred_1})
test_sub_1.to_csv('work/music_submit/submit_D_%.5f_01.csv'%(val_auc),index=False)

In [ ]:
del test_1
del test_id_1
del test_pred_1
gc.collect()
size_2=math.ceil(test.shape[0]*0.66)
test_2=test[size_1:size_2]
test_id_2=test_id[size_1:size_2]
test_pred_2=gbm_d.predict(test_2)
test_sub_2=pd.DataFram({'id':test_id_2,'target':test_pred_2})
test_sub_2.to_csv('work/music_submit/submit_D_%.5f_02.csv'%(val_auc),index=False)

In [ ]:
del test_2
del test_id_2
del test_pred_2
gc.collect()

test_3=test[size_2]
test_id_3=test_id[size_2:]d
test_pred_3=gbm_d.predict(test_3)
test_sub_3=pd.DataFrame({'id':test_id_3,'target':test_pred_3})
test_sub_3.to_csv('work/music_submit/submit_D_%.5f_03.csv'%(val_auc),index=False)

In [ ]:
test_sub=pd.concat([test_sub_1,test_sub_2,test_sub_3],axis=0)
test_sub.to_csv('work/music_submit/submit_D_%.5f_123.csv'%(val_auc),index=False)

In [ ]:
import pickle as cPickle
gbm_d=cPickle.load(open('work/music_lightgbm/submit_D.pkl','rb'))

读取四个训练好的模型预测的结果

In [ ]:
import gc
import math
import numpy as np
import pandas as pd


In [ ]:
a=pd.read_csv('work/music_submit/submit_A_%.5f_123.csv')
b=pd.read_csv('work/music_submit/submit_B_%.5f_123.csv')
c=pd.read_csv('work/music_submit/submit_C_%.5f_123.csv')
d=pd.read_csv('work/music_submit/submit_D_%.5f_123.csv')

In [ ]:
a1['target']=a['target']*0.6+b['target']*0.2+c['target']*0.1+d['target']*0.1
a1.to_csv('work/music_submit/submit',index=False)

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 